# Week 7, Day 5: Image Generation and Style Transfer

## Learning Objectives
- Understand generative models
- Learn style transfer techniques
- Master GAN architectures
- Practice implementing generators

## Topics Covered
1. Generative Models
2. Style Transfer
3. GANs
4. Image Synthesis

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose
from tensorflow.keras.applications import VGG19

## 1. Neural Style Transfer

In [ ]:
def style_transfer_example():
    # Load VGG19 model
    vgg = VGG19(include_top=False, weights='imagenet')
    
    # Content and style layers
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1',
                    'block2_conv1',
                    'block3_conv1',
                    'block4_conv1',
                    'block5_conv1']
    
    # Create model
    def get_model():
        vgg.trainable = False
        
        # Get outputs
        style_outputs = [vgg.get_layer(name).output for name in style_layers]
        content_outputs = [vgg.get_layer(name).output for name in content_layers]
        model_outputs = style_outputs + content_outputs
        
        return Model(vgg.input, model_outputs)
    
    # Load and preprocess images
    def load_img(path_to_img):
        img = tf.io.read_file(path_to_img)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, [224, 224])
        img = tf.expand_dims(img, axis=0)
        return img
    
    # Example usage
    content_path = 'content.jpg'
    style_path = 'style.jpg'
    
    content_image = load_img(content_path)
    style_image = load_img(style_path)
    
    # Display images
    plt.figure(figsize=(10, 5))
    
    plt.subplot(121)
    plt.imshow(content_image[0])
    plt.title('Content Image')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(style_image[0])
    plt.title('Style Image')
    plt.axis('off')
    
    plt.show()

style_transfer_example()

## 2. Generative Adversarial Networks

In [ ]:
def gan_example():
    # Generator model
    def make_generator():
        model = tf.keras.Sequential([
            Dense(7*7*256, use_bias=False, input_shape=(100,)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),
            
            tf.keras.layers.Reshape((7, 7, 256)),
            
            Conv2DTranspose(128, 5, strides=1, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),
            
            Conv2DTranspose(64, 5, strides=2, padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),
            
            Conv2DTranspose(1, 5, strides=2, padding='same', use_bias=False,
                           activation='tanh')
        ])
        return model
    
    # Discriminator model
    def make_discriminator():
        model = tf.keras.Sequential([
            Conv2D(64, 5, strides=2, padding='same',
                   input_shape=[28, 28, 1]),
            tf.keras.layers.LeakyReLU(),
            tf.keras.layers.Dropout(0.3),
            
            Conv2D(128, 5, strides=2, padding='same'),
            tf.keras.layers.LeakyReLU(),
            tf.keras.layers.Dropout(0.3),
            
            tf.keras.layers.Flatten(),
            Dense(1)
        ])
        return model
    
    # Create models
    generator = make_generator()
    discriminator = make_discriminator()
    
    # Generate sample images
    noise = tf.random.normal([16, 100])
    generated_images = generator(noise, training=False)
    
    # Display generated images
    plt.figure(figsize=(10, 10))
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.imshow(generated_images[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.show()

gan_example()

## 3. Image-to-Image Translation

In [ ]:
def pix2pix_example():
    # Generator
    def make_generator():
        inputs = Input(shape=[256, 256, 3])
        
        # Encoder
        down_stack = [
            Conv2D(64, 4, strides=2, padding='same'),
            Conv2D(128, 4, strides=2, padding='same'),
            Conv2D(256, 4, strides=2, padding='same'),
            Conv2D(512, 4, strides=2, padding='same'),
        ]
        
        # Decoder
        up_stack = [
            Conv2DTranspose(256, 4, strides=2, padding='same'),
            Conv2DTranspose(128, 4, strides=2, padding='same'),
            Conv2DTranspose(64, 4, strides=2, padding='same'),
        ]
        
        # Final layer
        last = Conv2DTranspose(3, 4, strides=2, padding='same')
        
        x = inputs
        
        # Downsampling
        skips = []
        for down in down_stack:
            x = down(x)
            skips.append(x)
        
        skips = reversed(skips[:-1])
        
        # Upsampling and establishing skip connections
        for up, skip in zip(up_stack, skips):
            x = up(x)
            x = tf.keras.layers.Concatenate()([x, skip])
        
        x = last(x)
        
        return Model(inputs=inputs, outputs=x)
    
    # Create model
    generator = make_generator()
    
    # Example usage
    sample_input = tf.random.normal([1, 256, 256, 3])
    generated_image = generator(sample_input, training=False)
    
    # Display results
    plt.figure(figsize=(10, 5))
    
    plt.subplot(121)
    plt.imshow(sample_input[0] * 0.5 + 0.5)
    plt.title('Input')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(generated_image[0] * 0.5 + 0.5)
    plt.title('Generated')
    plt.axis('off')
    
    plt.show()

pix2pix_example()

## Practical Exercises

In [ ]:
# Exercise 1: Style Transfer Implementation

def style_transfer_exercise():
    print("Task: Implement neural style transfer")
    print("1. Load content and style images")
    print("2. Extract features")
    print("3. Compute losses")
    print("4. Generate stylized image")
    
    # Your code here

style_transfer_exercise()

In [ ]:
# Exercise 2: Simple GAN

def gan_exercise():
    print("Task: Implement a basic GAN")
    print("1. Create generator")
    print("2. Create discriminator")
    print("3. Train models")
    print("4. Generate images")
    
    # Your code here

gan_exercise()

## MCQ Quiz

1. What is style transfer?
   - a) Image classification
   - b) Style application
   - c) Object detection
   - d) Image segmentation

2. What is a GAN?
   - a) Classification model
   - b) Generative model
   - c) Segmentation model
   - d) Detection model

3. What is the generator in GAN?
   - a) Classifier
   - b) Image creator
   - c) Feature extractor
   - d) Data loader

4. What is the discriminator in GAN?
   - a) Generator
   - b) Fake detector
   - c) Data loader
   - d) Image processor

5. What is content loss?
   - a) Training error
   - b) Feature difference
   - c) Model size
   - d) Image size

6. What is style loss?
   - a) Training error
   - b) Texture difference
   - c) Model size
   - d) Image size

7. What is mode collapse?
   - a) Model error
   - b) GAN problem
   - c) Training method
   - d) Loss function

8. What is cycle consistency?
   - a) Training method
   - b) Translation constraint
   - c) Loss function
   - d) Model architecture

9. What is perceptual loss?
   - a) Training error
   - b) Feature-based loss
   - c) Model size
   - d) Image size

10. What is progressive growing?
    - a) Training method
    - b) Resolution increase
    - c) Loss function
    - d) Model architecture

Answers: 1-b, 2-b, 3-b, 4-b, 5-b, 6-b, 7-b, 8-b, 9-b, 10-b